In [2]:
import pandas as pd
import datetime as dt
from datetime import timedelta
import os
import requests
import time

# Time how long this takes
start_time = time.time()

date = dt.date(2019, 3, 20)
while date <= dt.date(2019, 9, 29):
    print(date)
    
    # Check for if regular season games are played on this date
    response = requests.get('https://baseballsavant.mlb.com/schedule?date=%s' % (date,)).json()
    reg = False
    if response != []:
        for item in response["schedule"]["dates"]:
            for item2 in item["games"]:
                if item2["gameType"] == "R":
                    reg = True
    print(reg)
    # Only retrieve statcast data if there is 1+ regular season games
    if reg:
        yesterday = str(date)
        pitch_by_pitch = pd.read_csv('https://baseballsavant.mlb.com/statcast_search/csv?all=true&hfPT=&hfAB=&hfBBT=&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfGT=R%7C&hfC=&hfSea=2019%7C&hfSit=&player_type=batter&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt={}&game_date_lt={}&hfInfield=&team=&position=&hfOutfield=&hfRO=&home_road=&hfFlag=&hfPull=&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name-date&sort_col=xba&player_event_sort=h_launch_speed&sort_order=desc&min_pas=0&'.format(yesterday, yesterday)
                                 , parse_dates = ["game_date"]
                                 , index_col = ["player_id", "player_name"])
        """
        pitcher_name = pd.read_csv('https://baseballsavant.mlb.com/statcast_search/csv?all=true&hfPT=&hfAB=&hfBBT=&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfGT=R%7C&hfC=&hfSea=2019%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt={}&game_date_lt={}&hfInfield=&team=&position=&hfOutfield=&hfRO=&home_road=&hfFlag=&hfPull=&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name-date&sort_col=pitches&player_event_sort=h_launch_speed&sort_order=desc&min_pas=0&'.format(yesterday, yesterday)
                              , index_col = "pitcher")
        pitcher_name = pitcher_name[["player_name"]].rename(columns = {"player_name": "pitcher_name"})

        pitch_by_pitch = pitch_by_pitch.merge(pitcher_name, how = 'left', on = "pitcher")
        pitch_by_pitch.sort_values(["game_date", "game_pk", "inning", "at_bat_number", "pitch_number"], inplace = True)

        starting_pitchers = pitch_by_pitch[pitch_by_pitch["inning"] == 1].groupby(["game_pk", "inning_topbot"]).first()
        starting_pitchers.reset_index(inplace = True)
        starting_pitchers["starting_pitcher"] = True
        starting_pitchers = starting_pitchers[["pitcher", "starting_pitcher"]]

        pitch_by_pitch = pitch_by_pitch.merge(starting_pitchers, how = 'left', on = 'pitcher')
        pitch_by_pitch["starting_pitcher"].fillna(value = False, inplace = True)
        pitch_by_pitch.set_index("sv_id", inplace = True)
        """
        # if file does not exist write header 
        if not os.path.isfile("statcast.csv"):
            pitch_by_pitch.to_csv("statcast.csv", index = True)
        else: # else it exists so append without writing the header
            pitch_by_pitch.to_csv("statcast.csv", mode = 'a', header = False, index = True)

    date = date + timedelta(1)
    
print("--- %s minutes ---" % (time.time() - start_time) / 60)

2019-03-20
True
2019-03-21
True
2019-03-22
False
2019-03-23
False
2019-03-24
False
2019-03-25
False
2019-03-26
False
2019-03-27
False
2019-03-28
True
2019-03-29
True
2019-03-30
True
2019-03-31
True
2019-04-01
True
2019-04-02
True
2019-04-03
True
2019-04-04
True
2019-04-05
True
2019-04-06
True
2019-04-07
True
2019-04-08
True
2019-04-09
True
2019-04-10
True
2019-04-11
True
2019-04-12
True
2019-04-13
True
2019-04-14
True
2019-04-15
True
2019-04-16
True
2019-04-17
True
2019-04-18
True
2019-04-19
True
2019-04-20
True
2019-04-21
True
2019-04-22
True
2019-04-23
True
2019-04-24
True
2019-04-25
True
2019-04-26
True
2019-04-27
True
2019-04-28
True
2019-04-29
True
2019-04-30
True
2019-05-01
True
2019-05-02
True
2019-05-03
True
2019-05-04
True
2019-05-05
True
2019-05-06
True
2019-05-07
True
2019-05-08
True
2019-05-09
True
2019-05-10
True
2019-05-11
True
2019-05-12
True
2019-05-13
True
2019-05-14
True
2019-05-15
True
2019-05-16
True
2019-05-17
True
2019-05-18
True
2019-05-19
True
2019-05-20
True
20

TypeError: unsupported operand type(s) for /: 'str' and 'int'